# This Notebook is specifically to extract features from those chunks which we have made in other notebook

# This notenook should be run for the number of chunks you have made in parallel to speed up the process.

# For each chunk you have to change value of the chunk number only in the last cell of this notebook

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
np.random.seed(15)
import json
import shutil

# Make sure to mount your drive
You must have all the chunks of uri's from which you want to extract features from

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install spotipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 140 kB 7.5 MB/s 
     |████████████████████████████████| 246 kB 40.2 MB/s 
     |████████████████████████████████| 62 kB 1.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import time

In [ ]:
def get_track_features(track_id):
  audio = sp.audio_features(track_id)
  track = sp.track(track_id)

  # Album Features
  release_date = track['album']['release_date']
  album_total_tracks = track['album']['total_tracks']
  popularity = track['popularity']

  # Audio Features
  uri = audio[0]['id']
  danceability = audio[0]['danceability']
  energy = audio[0]['energy']
  key = audio[0]['key']
  loudness = audio[0]['loudness']
  mode = audio[0]['mode']
  speechiness = audio[0]['speechiness']
  acousticness = audio[0]['acousticness']
  instrumentalness = audio[0]['instrumentalness']
  liveness = audio[0]['liveness']
  valence = audio[0]['valence']
  tempo = audio[0]['tempo']
  time_signature = audio[0]['time_signature']

  track_data = [uri , release_date , album_total_tracks , popularity , danceability , energy , key , loudness , mode , speechiness , acousticness , instrumentalness , liveness , valence , tempo , time_signature]
  return track_data

In [ ]:
def Extract_Features(dataframe_chunk,chunk_no): 
  i=0    # counter variable 
  j=0   # variable to print the progress
  track_features=[]   # list to append the features recieved from sporify API
  for uri in dataframe_chunk:
    if i<900:   # if i < 900 (you can choose any counter number) this if condition will send 900 request for features to spotify one by one for each track uri
      
      if uri == '656TZlNdVe90zHvmebFt9U' or uri == '5GiU7GOYjDH2yp7fMf9w9j' or uri == '2ZF3gpgGPjOfi1yTWGX6Bm' or uri == '2odGdanrUsgu0d2i2Q5XoT' or uri == '6tf7ENqpF89Xox3xyNPGMv' or uri == '0fJ6UuZ7onqcldGr8HT5QV' or uri == '0Jb7g80rInlj3fa6W65Bp2':
        # this confition is to avoid getting error 404 since these uri's are not available anymore on spotify API
        track_features.append([uri if i==0 else np.nan for i in range(16)]) # for these uri's we are appending NaN values and will deal with these sample later

      else: # if other uri's (which we can extract features from spotify API)
        track_features.append(get_track_features(uri)) # storing recieved features from spotify API to later use them to concatenate in out main data frame
        j+=1 # increment j for every succesfully recieved features to keep the track of how many samples are done to later print it

    else: # if i is greater than 900, it means that we now have made enough request consequtively to API, so we will sleep for some time and reset the counter to 0
      time.sleep(0.1)
      print(j)
      i=0
    i+=1 # incrementing i for every uri in dataset
  uri_length = len(track_features)
  print("Number of total uri's we have completed extracting features: ", uri_length)
  track_features_df = pd.DataFrame(track_features, columns=['uri', 'release_date', 'album_total_tracks', 'popularity','danceability','energy','key','loudness', 'mode', 'speechiness','acousticness','instrumentalness','liveness','valence','tempo','time_signature'])
  track_features_df.to_csv("features("+chunk_no+").csv")
  shutil.copy("features("+chunk_no+").csv","/content/drive/MyDrive/Datasets")   # saving the dataframe chunk to drive

# Changing the credentials for while extracting features for each chunk may improve the speed of the process
It worked for me though :)

In [ ]:
auth_manager = SpotifyClientCredentials(client_id='6b92235a9748428bad27a3daf3df595a',
                                        client_secret='9b16b438472948a89671b1764825d3ad')
sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
chunk_num = 1   # change the number from whichever chunk you want tomextract features from

chunk = pd.read_csv('/content/drive/MyDrive/Datasets/chunk_'+str(chunk_num)+'.csv')
chunk.drop(columns='Unnamed: 0',inplace=True)
chunk = chunk.values.ravel()
Extract_Features(chunk,"Chunk_"+str(chunk_num))